In [1]:
import pandas as pd
import geopandas as gpd
from osm import OpenStreetMap
from shapely.geometry import Point, Polygon
from geo_utilities import decimal_to_km


import folium

In [2]:
osm = OpenStreetMap(osm_id='R148838') #United States

In [3]:
center = osm.get_geo_center_location()
center

(45.95349053726271, -113.25279009362077)

In [4]:
my_map = folium.Map(location=center, zoom_start=3, tiles="cartodb positron")
my_map

In [15]:
metadata = osm._get_metadata()
#metadata

In [6]:
geo_shape = osm.get_geo_shape()

In [7]:
data = pd.read_csv('./data.csv')
data.head()

/var/folders/q2/k9xdm7wn3_7cpc75plnlzl640000gn/T/ipykernel_36305/2826724597.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data.csv')


,Date_Of_Stop,Time_Of_Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,...,Article,Contributed_To_Accident,Race,Gender,Driver_City,Driver_State,DL_State,Arrest_Type,Day_of_Week,Hour_of_Day
0,2023-05-01,23:11:00,MCP,"3rd District, Silver Spring",OPERATING UNREGISTERED MOTOR VEHICLE ON HIGHWAY,BRIGGS CHANEY RD @ COLUMIBA PIKE,0.000000,0.000000,No,No,...,Transportation Article,False,WHITE,M,GAITHERSBURG,MD,MD,A - Marked Patrol,Monday,23
1,2023-04-30,23:55:00,MCP,"4th District, Wheaton",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,KEMP MILL ROAD @ ALPERT LANE,39.053038,-77.024637,No,No,...,Transportation Article,False,WHITE,F,SILVER SPRING,MD,MD,A - Marked Patrol,Sunday,23
2,2023-04-30,04:24:00,MCP,"2nd District, Bethesda",NEGLIGENT DRIVING VEHICLE IN CARELESS AND IMPR...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,No,...,Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,Sunday,4
3,2023-04-30,04:24:00,MCP,"2nd District, Bethesda",RECKLESS DRIVING VEHICLE IN WANTON AND WILLFUL...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,No,...,Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,Sunday,4
4,2023-04-30,04:24:00,MCP,"2nd District, Bethesda",FAILURE TO DRIVE VEHICLE ON RIGHT HALF OF ROAD...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,No,...,Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,Sunday,4


In [8]:
#data['geometry'] = data.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
#filtered_df = data[geo_shape.contains(data['geometry'])]
#filtered_df

# Clustering

In [9]:
coordinates = data[['Latitude', 'Longitude']].values

km = decimal_to_km(0.01)
km


1.1099999999999999

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from folium.plugins import MarkerCluster


random_sample = data.sample(frac=0.001, random_state=42).reset_index(inplace=False) 
print(random_sample.shape)
del random_sample['index']
random_sample.head()

(1857, 44)


,Date_Of_Stop,Time_Of_Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,...,Article,Contributed_To_Accident,Race,Gender,Driver_City,Driver_State,DL_State,Arrest_Type,Day_of_Week,Hour_of_Day
0,2019-01-01,01:17:00,MCP,Headquarters and Special Operations,POSSESSING SUSPENDED LIC.,CAMERON ST @ FENTON ST,38.997782,-77.027285,No,No,...,Transportation Article,False,BLACK,F,UPPER MARLBORO,MD,MD,A - Marked Patrol,Tuesday,1
1,2016-09-09,02:13:00,MCP,"6th District, Gaithersburg / Montgomery Village",HOLDER OF LEARNER'S PERMIT DRIVING WITH UN- AU...,MUDDY BRANCH / W DEER PARK,39.135957,-77.205783,No,No,...,Transportation Article,False,WHITE,M,ROCKVILLE,MD,MD,A - Marked Patrol,Friday,2
2,2018-01-31,22:56:00,MCP,"4th District, Wheaton",EXCEEDING POSTED MAXIMUM SPEED LIMIT: 98 MPH I...,GEORGIA AVE / SHOREFIELD RD,39.058487,-77.047972,No,No,...,Transportation Article,False,HISPANIC,M,SILVER SPRING,MD,MD,A - Marked Patrol,Wednesday,22
3,2017-10-05,15:29:00,MCP,"4th District, Wheaton",DRIVER FAILING TO GIVE TURN SIGNAL CONTINUOUSL...,IVAHAR ST/ HARRIS AVE,39.051437,-77.059165,No,No,...,Transportation Article,False,BLACK,M,WASHINGTON,DC,DC,A - Marked Patrol,Thursday,15
4,2019-10-11,05:44:00,MCP,"1st District, Rockville",THROWING ANY REFUSEON HWY,MD200 AT MILE 3.4,0.000000,0.000000,No,No,...,Transportation Article,False,WHITE,M,MILLERSVILLE,MD,MD,A - Marked Patrol,Friday,5


In [11]:
for index, row in random_sample.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(my_map)

In [12]:
my_map

In [14]:
center_lat = random_sample['Latitude'].mean()
center_lon = random_sample['Longitude'].mean()
map_center = [center_lat, center_lon]
my_map_random = folium.Map(location=map_center, zoom_start=13)

for index, row in random_sample.iterrows():
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                        radius=5,
                        color='blue',
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.6).add_to(my_map_random)
    
my_map_random

#### is there any spatial patterns in violations or arrests?



In [11]:

epsilon = 0.01  
min_samples = 5  

dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
data['cluster'] = dbscan.fit_predict(coordinates)

: 

In [ ]:
# Create a Folium map centered at the average latitude and longitude of your points
center_lat = df['latitud'].mean()
center_lon = df['longitud'].mean()
map_center = [center_lat, center_lon]
my_map = folium.Map(location=map_center, zoom_start=12)

# Create a MarkerCluster layer for better visualization of clustered points
marker_cluster = MarkerCluster().add_to(my_map)

# Add points to the map with different colors for each cluster
for index, row in df.iterrows():
    folium.Marker([row['latitud'], row['longitud']],
                  popup=f"Cluster: {row['cluster']}").add_to(marker_cluster)

# Save the map as an HTML file or display it
my_map.save('clustered_map.html')
# or
my_map


In [13]:
for index, row in data.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(my_map)

KeyboardInterrupt: 